In [1]:
import numpy as np
import cv2
import os
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp

In [10]:
path = 'C:\\Users\\james\\OneDrive\\Desktop\\VIT\\Sem III\\Image Processing\\Project\\Virus Preprocessed\\KD2'

In [11]:
p = Path(path)

def lout(check,neighbour):
    if(check > neighbour):
        return 0
    else:
        return 1
        
X=[]
y=[]

In [4]:
def vlbp(nimage):
    lbp = np.array(nimage)
    rows = lbp.shape[1]
    cols = lbp.shape[2]
    main=[]
    for i in range(1,3):
        for j in range(1,rows-1):
            for k in range(1,cols-1):
                check=lbp[i][j][k]
                #print(check)
                pattern=[]
                pattern.append(lout(check,lbp[i-1][j][k]))
                pattern.append(lout(check,lbp[i-1][j][k+1]))
                pattern.append(lout(check,lbp[i-1][j-1][k]))
                pattern.append(lout(check,lbp[i-1][j][k-1]))
                pattern.append(lout(check,lbp[i-1][j+1][k]))
                pattern.append(lout(check,lbp[i][j][k+1]))
                pattern.append(lout(check,lbp[i][j-1][k]))
                pattern.append(lout(check,lbp[i][j][k-1]))
                pattern.append(lout(check,lbp[i][j+1][k]))
                pattern.append(lout(check,lbp[i+1][j][k+1]))
                pattern.append(lout(check,lbp[i+1][j-1][k]))
                pattern.append(lout(check,lbp[i+1][j][k-1]))
                pattern.append(lout(check,lbp[i+1][j+1][k]))
                pattern.append(lout(check,lbp[i+1][j][k]))
                main.append(pattern)
    
    newimg=[]
    for item in main:
        temp=''.join(list(map(str,item)))
        a=int(str(temp), 2) #converting binary to integer. 
        newimg.append(a)
    
    test = np.histogram(newimg, bins=np.arange(0,2**14),range=(0,2**14), weights=None,density=1)
    X.append(test[0])

In [12]:
c=1
nimg = []
for x in p.iterdir():
        image16 = cv2.imread(str(x), cv2.IMREAD_GRAYSCALE)
        image16 = np.pad(image16, 1, mode='constant')
        
        if(c%4 != 0):
            nimg.append(image16)
        else:
            nimg.append(image16)
            vlbp(nimg)
            nimg=[]
            y.append(int((str(x).split("\\")[-1])[6:9]))


        #print(np.array(nimg).shape)
        c=c+1


In [13]:
X=np.array(X)
y=np.array(y)
y=y.reshape(len(y),1)

In [14]:
X.shape

(863, 16383)

In [15]:
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 18)


In [16]:
from sklearn import preprocessing as pr
X_train = pr.normalize(X_train,norm='l1')
X_test = pr.normalize(X_test,norm='l1')

In [17]:
from sklearn import svm
clftest = svm.SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [18]:
clftest.fit(X_train,y_train)

C:\Users\james\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [20]:
y_score = clftest.decision_function(X_test)

from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder()

enc.fit(y_test)

check = enc.transform(y_test).toarray()

nclasses=13
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(nclasses):
    fpr[i], tpr[i], _ = roc_curve(check[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

In [21]:
sum(roc_auc.values())/len(roc_auc.values())

0.6224671224155943